In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [45]:
# load data
# goods_info = 'data/data-trans/Goods Info May 27.csv'
# ec_info = 'data/data-trans/EcInfo conversion.csv'
# ticket_info = 'data/data-trans/Ticket Info May 27.csv'
# user_info = 'data/data-trans/User Info May 27.csv'

p_goods = pd.read_csv('data/data-trans/processed_goods.csv', low_memory=False)
p_ec = pd.read_csv('data/data-trans/processed_ec.csv', low_memory=False)
p_ticket = pd.read_csv('data/data-trans/processed_ticket.csv', low_memory=False)
p_user = pd.read_csv('data/data-trans/processed_user.csv', low_memory=False)

In [9]:
now = datetime.now()

### FC members

In [46]:
p_user.head()

,fc_member_id,season_id,bid,grade_cd,status,reception_class,apply_datetime,apply_class,price,admission_class,gender,birth
0,NaN,4,44476,J1,2,0,2019/07/08 22:19:43,0,2160,1,0,1982
1,8063.0,4,4142,G,0,1,2019/07/04 00:00:00,0,10800,1,1,1992
2,8065.0,4,46331,G,0,1,2019/07/04 00:00:00,0,10800,0,0,1978
3,8069.0,4,42328,G,0,1,2019/07/04 00:00:00,0,10800,1,0,1979
4,8070.0,4,4779,G,0,1,2019/07/04 00:00:00,0,10800,1,1,1991


In [47]:
p_user['purchased_date'] = pd.to_datetime(p_user['apply_datetime'], format='%Y/%m/%d %H:%M:%S')

#### RFM-FC

In [48]:
p_user.drop_duplicates()
current_season = p_user['season_id'].max()
p_user['monthly_recency'] = (now.year - p_user['purchased_date'].dt.year) * 12 + (now.month - p_user['purchased_date'].dt.month)


In [49]:
rfm_fc = p_user.groupby('bid').agg({
    # 'season_id': lambda x: current_season - x.max(),
    'monthly_recency': 'min', 
    'bid': 'count',
    'price': 'sum'
}).rename(columns={
    # 'season_id': 'recency_fc',
    'monthly_recency': 'recency_fc',
    'bid': 'frequency_fc',
    'price': 'monetary_fc'
}).reset_index()

In [81]:
rfm_fc['average_purchase_fc'] = rfm_fc['monetary_fc'] / rfm_fc['frequency_fc']

In [50]:
rfm_fc.head()

,bid,recency_fc,frequency_fc,monetary_fc
0,3553,48,2,6540
1,3558,11,4,20900
2,3560,11,4,64900
3,3561,23,4,68550
4,3562,14,5,24140


In [83]:
m_fc = p_user.merge(rfm_fc, on='bid', how='left').sort_values(by='bid').reset_index(drop=True)
m_fc.tail()

,fc_member_id,season_id,bid,grade_cd,status,reception_class,apply_class,price,admission_class,gender,birth,recency_fc,frequency_fc,monetary_fc,average_purchase_fc
37404,147818.0,8,148483,E,0,0,0,0,0,1,1978,0,1,0,0.0
37405,147910.0,8,148484,E,0,0,0,0,0,1,1996,0,1,0,0.0
37406,147871.0,8,148485,E,0,0,0,0,0,0,2017,0,1,0,0.0
37407,147894.0,8,148486,E,0,0,0,0,0,0,1969,0,1,0,0.0
37408,147930.0,8,148487,E,0,0,0,0,0,0,1970,0,1,0,0.0


In [51]:
rfm_fc.to_csv('data/data-trans/rfm_fc_monthly.csv', index=False)
# m_fc.to_csv('data/data-trans/final_fc.csv', index=False)

### Ticket

#### RFM-ticket

In [52]:
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format="%Y-%m-%d %H:%M:%S.%f")
    except ValueError:
        return pd.to_datetime(date_str, format="%Y-%m-%d %H:%M:%S")

# Apply the custom parser function to the 'order_date' column
p_ticket['order_date'] = p_ticket['order_date'].apply(parse_date)



In [53]:
p_ticket['order_date'] = pd.to_datetime(p_ticket['order_date'])

In [54]:
# p_ticket[p_ticket['season_id'].isna()].tail()

p_ticket.drop_duplicates()

,ticket_id,event_id,event_date,order_id,order_date,outlet,quantity,price_code,price_level,price,is_cancel,bid,fc_member_id,status,fc_status,season_id,grade_cd,gender,birth,reception_class
0,694705877:94262017,694705877:298,2023-03-25 15:05:00+00:00,1000000000808255,2023-03-24 20:54:41.560,FLASH,1,NaN,NaN,0.0,False,121253.0,NaN,0.0,NaN,NaN,NaN,0.0,1975.0,NaN
1,694705877:94760790,694705877:365,2024-05-11 19:05:00+00:00,1000000001438699,2024-04-15 21:11:02.000,FLASH,1,NaN,NaN,0.0,False,125653.0,NaN,0.0,NaN,NaN,NaN,0.0,1995.0,NaN
2,694705877:94571934,694705877:347,2024-01-17 19:35:00+00:00,1000000001209635,2024-01-17 07:51:39.000,FLASH,1,NaN,NaN,0.0,False,62264.0,NaN,0.0,NaN,NaN,NaN,0.0,1988.0,NaN
3,694705877:94475945,694705877:333,2023-11-08 19:35:00+00:00,1000000001049130,2023-11-06 17:12:04.390,FLASH,1,NaN,NaN,0.0,False,96146.0,NaN,0.0,NaN,NaN,NaN,0.0,1992.0,NaN
4,694705877:94706561,694705877:361,2024-04-17 19:35:00+00:00,1000000001363625,2024-03-20 17:33:33.000,FLASH,1,NaN,NaN,0.0,False,117406.0,NaN,0.0,NaN,NaN,NaN,0.0,1965.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790861,694705877:94790258,694705877:364,2024-05-10 19:05:00+00:00,1110513873,2024-04-30 11:29:33.000,Internet,1,【招待】RS SNS大人,4F自由席,0.0,False,55420.0,33306.0,0.0,0.0,8.0,R1,1.0,1973.0,0.0
790862,694705877:94790177,694705877:364,2024-05-10 19:05:00+00:00,1110513845,2024-04-30 06:56:55.000,Internet,1,【招待】RS SNSこども,4F自由席,0.0,False,50137.0,40266.0,0.0,0.0,8.0,E,0.0,1972.0,0.0
790863,694705877:94790176,694705877:364,2024-05-10 19:05:00+00:00,1110513845,2024-04-30 06:56:55.000,Internet,1,【招待】RS SNS大人,4F自由席,0.0,False,50137.0,40266.0,0.0,0.0,8.0,E,0.0,1972.0,0.0
790864,694705877:94790108,694705877:364,2024-05-10 19:05:00+00:00,1110513824,2024-04-30 00:38:36.000,Internet,1,【招待】RS SNS大人,4F自由席,0.0,False,54918.0,40553.0,0.0,0.0,8.0,E,0.0,1983.0,0.0


In [55]:
from datetime import datetime, date
def map_season(order_date):
    order_date = order_date.date()
    if date(2016, 7, 1) <= order_date <= date(2017, 6, 30):
        return 1
    elif date(2017, 7, 1) <= order_date <= date(2018, 6, 30):
        return 2
    elif date(2018, 7, 1) <= order_date <= date(2019, 6, 30):
        return 3
    elif date(2019, 7, 1) <= order_date <= date(2020, 6, 30):
        return 4
    elif date(2020, 7, 1) <= order_date <= date(2021, 6, 30):
        return 5
    elif date(2021, 7, 1) <= order_date <= date(2022, 6, 30):
        return 6
    elif date(2022, 7, 1) <= order_date <= date(2023, 6, 30):
        return 7
    elif date(2023, 7, 1) <= order_date <= date(2024, 6, 30):
        return 8



p_ticket['mapped_season_id'] = p_ticket['order_date'].apply(map_season)
current_season = p_ticket['mapped_season_id'].max()

In [56]:

p_ticket['monthly_recency'] = (now.year - p_ticket['order_date'].dt.year) * 12 + (now.month - p_ticket['order_date'].dt.month)

In [57]:
p_ticket.head()

,ticket_id,event_id,event_date,order_id,order_date,outlet,quantity,price_code,price_level,price,...,fc_member_id,status,fc_status,season_id,grade_cd,gender,birth,reception_class,mapped_season_id,monthly_recency
0,694705877:94262017,694705877:298,2023-03-25 15:05:00+00:00,1000000000808255,2023-03-24 20:54:41.560,FLASH,1,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,1975.0,NaN,7,16
1,694705877:94760790,694705877:365,2024-05-11 19:05:00+00:00,1000000001438699,2024-04-15 21:11:02.000,FLASH,1,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,1995.0,NaN,8,3
2,694705877:94571934,694705877:347,2024-01-17 19:35:00+00:00,1000000001209635,2024-01-17 07:51:39.000,FLASH,1,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,1988.0,NaN,8,6
3,694705877:94475945,694705877:333,2023-11-08 19:35:00+00:00,1000000001049130,2023-11-06 17:12:04.390,FLASH,1,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,1992.0,NaN,8,8
4,694705877:94706561,694705877:361,2024-04-17 19:35:00+00:00,1000000001363625,2024-03-20 17:33:33.000,FLASH,1,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,0.0,1965.0,NaN,8,4


In [58]:
rfm_t = p_ticket.groupby('bid').agg({
    # 'mapped_season_id': lambda x: current_season - x.max(),
    'monthly_recency': 'min', 
    'bid': 'count',
    'price': 'sum'
}).rename(columns={
    # 'mapped_season_id': 'recency_t',
    'monthly_recency': 'recency_t',
    'bid': 'frequency_t',
    'price': 'monetary_t'
}).reset_index()

In [90]:
rfm_t['average_purchase_t'] = rfm_t['monetary_t'] / rfm_t['frequency_t']

In [59]:
# rfm_t[rfm_t['bid']==49880]
rfm_t.head()

,bid,recency_t,frequency_t,monetary_t
0,3553.0,10,6,39600.0
1,3554.0,17,7,9000.0
2,3558.0,5,125,1158800.0
3,3559.0,20,1,5000.0
4,3560.0,4,56,504600.0


In [92]:
m_ticket = p_ticket.merge(rfm_t, on='bid', how='left').sort_values(by='bid').reset_index(drop=True)
m_ticket.tail()

,ticket_id,event_id,event_date,order_id,order_date,outlet,quantity,price_code,price_level,price,...,season_id,grade_cd,gender,birth,reception_class,mapped_season_id,recency_t,frequency_t,monetary_t,average_purchase_t
790861,694705877:94754017,694705877:365,2024-05-11 19:05:00+00:00,1110503457,2024-04-13 20:09:52,Internet,1,【DP】共通 / 大人,1Fベンチ側 前方,10500.0,...,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
790862,694705877:94754016,694705877:365,2024-05-11 19:05:00+00:00,1110503457,2024-04-13 20:09:52,Internet,1,【DP】共通 / 大人,1Fベンチ側 前方,10500.0,...,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
790863,694705877:94784762,694705877:369,2024-05-21 19:05:00+00:00,1110512304,2024-04-27 20:02:20,Internet,1,【DP】共通 / 大人,コートエンド1列 ホーム,20000.0,...,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
790864,694705877:94627343,694705877:360,2024-03-24 15:05:00+00:00,1110471575,2024-01-20 20:23:02,Internet,0,【前売】共通,ペアシート HOME側 12番,5000.0,...,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
790865,694705877:94689760,694705877:358,2024-04-14 15:05:00+00:00,1110485973,2024-02-27 20:18:32,Internet,0,【前売】大人,【特典】スタンドA Mブロック,22000.0,...,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN


In [60]:
rfm_t.to_csv('data/data-trans/rfm_t_monthly.csv', index=False)
# m_ticket.to_csv('data/data-trans/final_ticket.csv', index=False)

### EC

#### RFM-EC

In [61]:
p_ec.head()
p_ec_t = p_ec.drop(columns=['product_id','quantity'])
p_ec_t.drop_duplicates()

,order_date,order_id,total_amount,fc_member_id,bid,purchase_season,status,fc_status,gender,birth,grade_cd
0,2019/07/08,8,2600,NaN,5636,4,2,NaN,0,1969,NaN
3,2020/08/01,50476,8365,NaN,5636,5,2,NaN,0,1969,NaN
6,2020/08/01,50477,2665,NaN,5636,5,2,NaN,0,1969,NaN
7,2019/11/13,5755,4215,NaN,4972,4,0,NaN,1,1995,NaN
9,2020/01/01,11064,11115,NaN,4972,4,0,NaN,1,1995,NaN
...,...,...,...,...,...,...,...,...,...,...,...
24900,2023/10/11,246145,46765,3540.0,7693,8,2,2.0,1,1998,E
24903,2023/10/29,249811,3065,3540.0,7693,8,2,2.0,1,1998,E
24904,2023/11/16,253015,3965,3540.0,7693,8,2,2.0,1,1998,E
24907,2023/12/08,257359,12765,3540.0,7693,8,2,2.0,1,1998,E


In [62]:
# current_season = p_ec['purchase_season'].max()
p_ec_t['order_date'] = pd.to_datetime(p_ec_t['order_date'], format='%Y/%m/%d')
p_ec_t['monthly_recency'] = (now.year - p_ec_t['order_date'].dt.year) * 12 + (now.month - p_ec_t['order_date'].dt.month)

In [63]:
p_ec_t.head()

,order_date,order_id,total_amount,fc_member_id,bid,purchase_season,status,fc_status,gender,birth,grade_cd,monthly_recency
0,2019-07-08,8,2600,NaN,5636,4,2,NaN,0,1969,NaN,60
1,2019-07-08,8,2600,NaN,5636,4,2,NaN,0,1969,NaN,60
2,2019-07-08,8,2600,NaN,5636,4,2,NaN,0,1969,NaN,60
3,2020-08-01,50476,8365,NaN,5636,5,2,NaN,0,1969,NaN,47
4,2020-08-01,50476,8365,NaN,5636,5,2,NaN,0,1969,NaN,47


In [64]:
rfm_ec = p_ec_t.groupby('bid').agg({
    # 'purchase_season': lambda x: current_season - x.max(),
    'monthly_recency': 'min', 
    'bid': 'count',
    'total_amount': 'sum'
}).rename(columns={
    # 'purchase_season': 'recency_ec',
    'monthly_recency': 'recency_ec',
    'bid': 'frequency_ec',
    'total_amount': 'monetary_ec'
}).reset_index()

In [97]:
rfm_ec['average_purchase_ec'] = rfm_ec['monetary_ec'] / rfm_ec['frequency_ec']

In [65]:
rfm_ec

,bid,recency_ec,frequency_ec,monetary_ec
0,3551,30,5,52425
1,3552,30,23,283125
2,3553,37,11,63395
3,3554,38,2,7306
4,3555,5,7,19805
...,...,...,...,...
4804,147116,3,2,7530
4805,147117,2,3,8195
4806,147118,3,2,89530
4807,147119,2,1,2765


In [99]:
m_ec = p_ec.merge(rfm_ec, on='bid', how='left').sort_values(by='bid').reset_index(drop=True)
m_ec.head(10)

,order_date,order_id,product_id,quantity,total_amount,fc_member_id,bid,purchase_season,status,fc_status,gender,birth,grade_cd,recency_ec,frequency_ec,monetary_ec,average_purchase_ec
0,2021/10/09,119640,46241,1,3765,NaN,3551,6,0,NaN,0,1978,NaN,2,5,52425,10485.000000
1,2021/12/16,130431,100941,1,5665,NaN,3551,6,0,NaN,0,1978,NaN,2,5,52425,10485.000000
2,2022/01/02,133351,101882,1,20765,NaN,3551,6,0,NaN,0,1978,NaN,2,5,52425,10485.000000
3,2020/03/20,20456,46242,1,11115,NaN,3551,4,0,NaN,0,1978,NaN,2,5,52425,10485.000000
4,2020/03/20,20456,46255,1,11115,NaN,3551,4,0,NaN,0,1978,NaN,2,5,52425,10485.000000
5,2020/04/11,26129,39772,1,5357,NaN,3552,4,0,NaN,1,1983,NaN,2,23,283125,12309.782609
6,2020/04/11,26129,21997,1,5357,NaN,3552,4,0,NaN,1,1983,NaN,2,23,283125,12309.782609
7,2020/04/11,26129,47714,1,5357,NaN,3552,4,0,NaN,1,1983,NaN,2,23,283125,12309.782609
8,2020/06/09,40978,47907,1,4645,105.0,3552,4,0,NaN,1,1983,NaN,2,23,283125,12309.782609
9,2020/06/09,40978,23640,1,4645,105.0,3552,4,0,NaN,1,1983,NaN,2,23,283125,12309.782609


In [66]:
rfm_ec.to_csv('data/data-trans/rfm_ec_monthly.csv', index=False)
# m_ec.to_csv('data/data-trans/final_ec.csv', index=False)

### Final

In [101]:
m_fc.drop_duplicates()
m_ticket.drop_duplicates()
m_ec.drop_duplicates()

,order_date,order_id,product_id,quantity,total_amount,fc_member_id,bid,purchase_season,status,fc_status,gender,birth,grade_cd,recency_ec,frequency_ec,monetary_ec,average_purchase_ec
0,2021/10/09,119640,46241,1,3765,NaN,3551,6,0,NaN,0,1978,NaN,2,5,52425,10485.000000
1,2021/12/16,130431,100941,1,5665,NaN,3551,6,0,NaN,0,1978,NaN,2,5,52425,10485.000000
2,2022/01/02,133351,101882,1,20765,NaN,3551,6,0,NaN,0,1978,NaN,2,5,52425,10485.000000
3,2020/03/20,20456,46242,1,11115,NaN,3551,4,0,NaN,0,1978,NaN,2,5,52425,10485.000000
4,2020/03/20,20456,46255,1,11115,NaN,3551,4,0,NaN,0,1978,NaN,2,5,52425,10485.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24904,2024/04/21,292849,152072,1,2615,146989.0,147117,8,0,0.0,0,1965,E,0,3,8195,2731.666667
24905,2024/04/16,291861,138801,1,44765,NaN,147118,8,0,NaN,0,2001,NaN,0,2,89530,44765.000000
24906,2024/04/16,291861,138800,1,44765,NaN,147118,8,0,NaN,0,2001,NaN,0,2,89530,44765.000000
24907,2024/05/10,297845,66596,1,2765,NaN,147119,8,0,NaN,0,1987,NaN,0,1,2765,2765.000000


In [102]:
# m_fc.drop(columns=['average_purchase'], inplace=True)
# m_ticket.drop(columns=['average_purchase'], inplace=True)
# m_ec.drop(columns=['average_purchase'], inplace=True)

In [103]:
# rfm_fc.drop(columns=['average_purchase'], inplace=True)
# rfm_t.drop(columns=['average_purchase'], inplace=True)
# rfm_ec.drop(columns=['average_purchase'], inplace=True)

In [104]:
rfm_fc.to_csv('data/data-trans/rfm_fc.csv', index=False)
rfm_t.to_csv('data/data-trans/rfm_t.csv', index=False)
rfm_ec.to_csv('data/data-trans/rfm_ec.csv', index=False)

m_fc.to_csv('data/data-trans/final_fc.csv', index=False)
m_ticket.to_csv('data/data-trans/final_ticket.csv', index=False)
m_ec.to_csv('data/data-trans/final_ec.csv', index=False)

In [78]:
m_rfm = pd.merge(rfm_t, rfm_ec, on='bid', how='outer')
m_rfm = pd.merge(m_rfm, rfm_fc, on='bid', how='outer')
m_rfm.drop_duplicates()

,bid,recency_t,frequency_t,monetary_t,recency_ec,frequency_ec,monetary_ec,recency_fc,frequency_fc,monetary_fc
0,3551.0,NaN,NaN,NaN,30.0,5.0,52425.0,NaN,NaN,NaN
1,3552.0,NaN,NaN,NaN,30.0,23.0,283125.0,NaN,NaN,NaN
2,3553.0,10.0,6.0,39600.0,37.0,11.0,63395.0,48.0,2.0,6540.0
3,3554.0,17.0,7.0,9000.0,38.0,2.0,7306.0,NaN,NaN,NaN
4,3555.0,NaN,NaN,NaN,5.0,7.0,19805.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
123165,161739.0,22.0,1.0,3500.0,NaN,NaN,NaN,NaN,NaN,NaN
123166,161740.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
123167,161741.0,4.0,2.0,7800.0,NaN,NaN,NaN,NaN,NaN,NaN
123168,161742.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:

m_rfm.to_csv('data/data-trans/final_rfm_monthly.csv', index=False)

In [76]:
rfm_ec['recency_ec'].max()

60